In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/My Drive/ML team project/Project 2/Neubauer Chamber Automation

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/12swurGjFVTSgi6fpziRcJ-tqsj6OoBgK/ML team project/Project 2/Neubauer Chamber Automation


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Resize

import numpy as np
import cv2
import time
import os

import matplotlib.pyplot as plt

## Instagram Dataset
The following class is used to load in the Instagram Neubauer images along with the the corresponding masks

In [ ]:
NEU_CELL_AUG_PATH = 'Data Set/train_augmented/images/'
NEU_MASK_AUG_PATH = 'Data Set/train_augmented/masks/'

class NeubauerDataset(Dataset):
    def __init__(self):
        #Get the image paths. By sorting we ensure that the image and mask belong together
        img_paths = sorted(os.listdir(NEU_CELL_AUG_PATH))
        mask_paths = sorted(os.listdir(NEU_MASK_AUG_PATH))
        self.img_paths = [NEU_CELL_AUG_PATH+filename for filename in img_paths if '.jpg' in filename]
        self.mask_paths = [NEU_MASK_AUG_PATH+filename for filename in mask_paths if '.jpg' in filename]

        #Some images are quite large, or have different sizes. Here we resize them to a suitable shape for the UNet
        self.resize = Resize((512,512))
    
    #Create the mask classes. The masks we load in will have a shape of 1x512x512
    #however, we need 2x512x512 (one channel for the background, and one channel for the cells themselves)
    def create_mask_labels(self, mask):
        background = np.ones_like(mask)
        background[mask==1] = 0

        cell_mask = np.ones_like(mask)
        cell_mask[mask==0] = 0

        return np.dstack((background, cell_mask))

    #Apply a random mask
    def random_mask(self, image, mask):
        #Create one large mask randomly in the image
        if np.random.random() <= 0.5:
            i = np.random.randint(128,384)
            j = np.random.randint(128,384)
            r = 64
            image[:, i-r:i+r, j-r:j+r] = 0
            mask[0, i-r:i+r, j-r:j+r] = 1
            mask[1, i-r:i+r, j-r:j+r] = 0
            return image, mask
        #OR, create 4 smaller masks randomly in the image
        for _ in range(4):
            i = np.random.randint(64,448)
            j = np.random.randint(64,448)
            r = 48
            image[:, i-r:i+r, j-r:j+r] = 0
            mask[0, i-r:i+r, j-r:j+r] = 1
            mask[1, i-r:i+r, j-r:j+r] = 0
        return image, mask
    
    def __getitem__(self, i):
        #Load the images and masks
        cell_path = self.img_paths[i]
        mask_path = self.mask_paths[i]
        image = cv2.imread(cell_path)/255.0
        #Get the shape and data type to the correct form for the UNet
        image = torch.from_numpy(image.astype('float32')).transpose(2,0)
        mask = cv2.imread(mask_path, 0)

        #Create the proper mask labels, and get the shape to the correct form
        mask = self.create_mask_labels(mask/255.0)
        mask = torch.from_numpy(mask.astype('float32')).transpose(2,0)
        
        #Resize the images to be 512x512. We threshold the mask again since 
        #resizing interpolates some of the values which would lead to a non-binary mask
        mask = self.resize(mask)
        mask[mask<=0.5] = 0.0
        mask[mask>0.5] = 1.0
        image = self.resize(image)

        #Randomly apply a random mask to the image with probability 0.25
        if np.random.random() <= 0.25:
            image, mask = self.random_mask(image, mask)
        return image, mask

    def __len__(self):
        return len(self.img_paths)
#dataset = NeubauerDataset()
#dataloader = DataLoader(dataset, batch_size=16, shuffle=False, drop_last=True)
#iterator = iter(dataloader)
